<a href="https://colab.research.google.com/github/ArthurChan-1111/Bitcoin_prediction/blob/main/Master_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

%cd Bitcoin_prediction

import pandas as pd
import numpy as np

df_bitcoin_by_day = pd.read_csv('btc_1d_data_2018_to_2025.csv', on_bad_lines='skip', lineterminator='\n')


正複製到 'Bitcoin_prediction'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
接收物件中: 100% (21/21), 173.47 KiB | 389.00 KiB/s, 完成.
處理 delta 中: 100% (6/6), 完成.
/Users/arthurchan/Documents/STAT5102/Project/Bitcoin_prediction/Bitcoin_prediction


In [2]:
df_bitcoin_by_day.describe().T

,count,mean,std,min,25%,50%,75%,max
Open,2638.0,3.007471e+04,2.462686e+04,3.211710e+03,9.141495e+03,2.316573e+04,4.456562e+04,1.061438e+05
High,2638.0,3.076510e+04,2.511586e+04,3.276500e+03,9.304870e+03,2.373610e+04,4.616359e+04,1.083530e+05
Low,2638.0,2.933413e+04,2.411405e+04,3.156260e+03,8.874750e+03,2.270910e+04,4.336132e+04,1.053215e+05
Close,2638.0,3.010409e+04,2.465340e+04,3.211720e+03,9.141368e+03,2.317984e+04,4.466510e+04,1.061438e+05
Volume,2638.0,6.986922e+04,8.017029e+04,4.811771e+02,2.935439e+04,4.436260e+04,7.258744e+04,7.607054e+05
Quote asset volume,2638.0,1.806400e+09,2.007644e+09,1.177017e+07,3.825199e+08,1.126328e+09,2.526423e+09,1.746531e+10
Number of trades,2638.0,1.740156e+06,2.117254e+06,1.241700e+04,4.140775e+05,1.018162e+06,1.931050e+06,1.522359e+07
Taker buy base asset volume,2638.0,3.472933e+04,3.989757e+04,2.313948e+02,1.478038e+04,2.222689e+04,3.584608e+04,3.747756e+05
Taker buy quote asset volume,2638.0,8.943653e+08,9.983054e+08,6.532639e+06,1.907035e+08,5.474179e+08,1.260436e+09,8.783916e+09
Ignore,2638.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [3]:
def calculate_indicators(df):
    # Calculate RSI
    def rsi(series, period=14):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    # Calculate CCI
    def cci(series, period=14):
        typical_price = (series['High'] + series['Low'] + series['Close']) / 3
        ma = typical_price.rolling(window=period).mean()
        mad = (typical_price - ma).abs().rolling(window=period).mean()
        return (typical_price - ma) / (0.015 * mad)

    # Calculate Bollinger Bands
    def bollinger_bands(series, window=20, num_std_dev=2):
        sma = series['Close'].rolling(window=window).mean()
        rstd = series['Close'].rolling(window=window).std()
        upper_band = sma + (rstd * num_std_dev)
        lower_band = sma - (rstd * num_std_dev)
        return upper_band, lower_band

    # Calculating indicators
    df['RSI7'] = rsi(df['Close'], period=7)
    df['RSI14'] = rsi(df['Close'], period=14)
    df['CCI7'] = cci(df, period=7)
    df['CCI14'] = cci(df, period=14)
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['Bollinger_Upper'], df['Bollinger_Lower'] = bollinger_bands(df)

    #Cup and handle signal
    #To be completed
    #/Cup and handle signal

    # Set Binary Signal
    df['Signal_Up'] = 0

    # Use .loc for assignment to avoid warnings
    df.loc[1:, 'Signal_Up'] = np.where(df['Close'].iloc[1:].values > df['Close'].iloc[:-1].values, 1, 0)


    # Add a column to indicate if the price rose by 5% or more compared to the previous day
    df_bitcoin_by_day['Signal_5Percent_Up'] = 0  # Initialize with 0

    # Use .loc for assignment to avoid warnings
    df_bitcoin_by_day.loc[1:, 'Signal_5Percent_Up'] = np.where(
        (df_bitcoin_by_day['Close'].iloc[1:].values >= df_bitcoin_by_day['Close'].iloc[:-1].values * 1.05),
        1,
        0
    )
    return df

calculate_indicators(df_bitcoin_by_day)

df_bitcoin_by_day.tail()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,Ignore,RSI7,RSI14,CCI7,CCI14,MA50,Bollinger_Upper,Bollinger_Lower,Signal_Up,Signal_5Percent_Up
2633,2025-03-18,84010.02,84021.74,83017.88,83018.33,1110.53302,2025-03-18 23:59:59.999,9.266588e+07,179190,436.70291,...,0,68.363371,41.512263,3.070044,-29.091901,92459.5592,92228.296511,78262.968489,0,0
2634,2025-03-19,82715.03,83280.00,82603.20,83010.87,1360.52066,2025-03-19 23:59:59.999,1.129231e+08,209016,829.38438,...,0,46.272092,33.562588,-25.112016,-27.920901,92093.0662,92100.209554,78046.600446,0,0
2635,2025-03-20,86845.93,87453.67,85733.11,85905.91,3146.08729,2025-03-20 23:59:59.999,2.723224e+08,399959,1421.72638,...,0,61.212357,41.838930,151.593782,67.947207,91736.5196,92177.214055,78294.227945,1,0
2636,2025-03-21,84223.38,84748.44,84180.09,84711.21,656.96798,2025-03-21 23:59:59.999,5.548234e+07,117589,341.94482,...,0,69.261525,45.401429,29.070907,27.628368,91336.2850,92181.597884,78303.586116,0,0
2637,2025-03-22,84088.79,84338.98,83989.58,84037.82,534.27259,2025-03-22 23:59:59.999,4.496609e+07,86830,231.88751,...,0,48.410583,44.864757,4.511839,21.414159,90968.4502,90226.130479,79235.835521,0,0


In [5]:
df_bitcoin_by_day[df_bitcoin_by_day['Signal_Up']==1]

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,Ignore,RSI7,RSI14,CCI7,CCI14,MA50,Bollinger_Upper,Bollinger_Lower,Signal_Up,Signal_5Percent_Up
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2018-01-02 23:59:59.999,2.797171e+08,177728,11346.326739,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2018-01-03 23:59:59.999,2.361169e+08,162787,8994.953566,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,2018-01-04 23:59:59.999,3.127816e+08,170310,12680.812951,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,2018-01-05 23:59:59.999,3.693220e+08,192969,13346.622293,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
5,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,2018-01-06 23:59:59.999,3.092169e+08,158242,11007.164056,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,2025-03-05,87281.98,90933.82,86334.53,90283.57,36649.484150,2025-03-05 23:59:59.999,3.254229e+09,7716640,18359.331920,...,0,60.149200,44.278219,40.253500,-23.914560,97367.9326,103117.912039,81965.731961,1,0
2627,2025-03-12,82932.99,83591.07,82604.95,83488.72,1624.191460,2025-03-12 23:59:59.999,1.349775e+08,311866,977.529250,...,0,28.378680,49.162174,-32.037434,-39.030067,94897.7358,99531.739980,77298.730020,1,1
2628,2025-03-13,83680.12,84000.00,83312.92,83915.10,1854.128840,2025-03-13 23:59:59.999,1.552110e+08,273157,940.659960,...,0,30.945065,47.090678,-0.568450,-32.759954,94501.9046,97947.414677,77444.565323,1,0
2630,2025-03-15,83983.19,84583.04,83964.57,84227.92,782.819720,2025-03-15 23:59:59.999,6.596448e+07,131694,425.748270,...,0,43.619037,49.595132,40.758207,-29.557826,93644.6198,95043.867155,77656.577845,1,0


In [6]:
df_bitcoin_by_day[df_bitcoin_by_day['Signal_5Percent_Up']==1]

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,Ignore,RSI7,RSI14,CCI7,CCI14,MA50,Bollinger_Upper,Bollinger_Lower,Signal_Up,Signal_5Percent_Up
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2018-01-02 23:59:59.999,2.797171e+08,177728,11346.326739,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,2018-01-05 23:59:59.999,3.693220e+08,192969,13346.622293,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
19,2018-01-20,11474.98,13099.00,11412.45,12799.94,28768.857827,2018-01-20 23:59:59.999,3.585611e+08,251647,14389.196005,...,0,36.928057,30.954953,28.910079,NaN,NaN,17560.179217,10215.159783,1,1
23,2018-01-24,10799.14,11570.48,10500.00,11349.99,27158.587762,2018-01-24 23:59:59.999,2.994706e+08,293429,13087.393088,...,0,54.018137,33.320079,-19.936438,NaN,NaN,17383.720292,9032.124708,1,1
36,2018-02-06,6939.63,7878.00,6000.01,7652.14,100201.500307,2018-02-06 23:59:59.999,6.843643e+08,785996,49565.020794,...,0,22.835390,30.340584,-69.165579,-142.530181,NaN,13417.628766,7298.310234,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,2024-11-06,69372.01,76400.00,69298.00,75571.99,104126.994787,2024-11-06 23:59:59.999,7.702283e+09,8434184,53243.884427,...,0,63.208343,70.537266,165.962242,188.375440,65715.1014,73606.891524,64702.842476,1,1
2506,2024-11-11,80370.01,89530.54,80216.01,88647.99,82323.665776,2024-11-11 23:59:59.999,6.944269e+09,9348890,42837.295314,...,0,100.000000,82.829760,133.646642,220.988960,67379.6290,83108.094125,60943.803875,1,1
2577,2025-01-21,102260.00,107240.81,100119.04,106143.82,45941.020020,2025-01-21 23:59:59.999,4.768512e+09,6842980,23227.027410,...,0,74.440516,66.180939,48.817859,136.925247,98354.8208,106095.737591,90920.631409,1,1
2617,2025-03-02,86064.54,95000.00,85050.60,94270.00,54889.090450,2025-03-02 23:59:59.999,4.972550e+09,7403218,29273.810310,...,0,46.374818,47.460192,41.397778,-17.564925,97805.0498,104169.665347,83781.596653,1,1
